In [ ]:
%load_ext autoreload
%autoreload 2

## 이전 작업에서 생성한 job_name을 `job_name` 변수에 입력하시기 바랍니다.

- `job_name`

In [ ]:
## From 1st notebook
job_name="YOUR_JOBNAME" 

job_name_prefix = 'sagemaker-imageclassification-cifar10'

## Model name variable
model_name="image-classification-cifar-transfer"

## Training한 Model을 배포 하기 위해서는 크게 3단계 작업이 필요합니다.

1. Model 생성
    - docker 이미지
    - model 데이터
1. Endpoint 설정 파일 생성
1. Endpoint 생성

> [AWS Sagemaker 문서](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/ex1-deploy-model.html)

In [ ]:
import boto3
sage = boto3.Session().client(service_name='sagemaker') 
from sagemaker import get_execution_role
role = get_execution_role()

## Model name이 있는지 확인

In [ ]:
## Check model_name
if model_name in [x['ModelName'] for x in sage.list_models()['Models']]:
    print("{} already exists!".format(model_name))
    print("Deleting {}".format(model_name))
    sage.delete_model(ModelName = model_name)

## Model 생성에 필요한 container 및 model data 준비

In [ ]:
info = sage.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:latest'}
hosting_image = containers[boto3.Session().region_name]
primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

## Model 생성

In [ ]:
create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)
print(create_model_response['ModelArn'])

## Endpoint 설정 파일 생성


A/B 테스트 목적으로 동시에 여러 모델을 호스팅 할 때 필요한 REST endpoints를 구성하는 것을 지원합니다. 이를 지원하기 위해 고객이 endpoint 설정 파일을 생성하고 트래픽을 어떻게 분할, 분배, shadow, 샘플링을 할지를 설정 합니다.

또한 endpoint 설정에서 모델 배포에 필요한 인스턴스 유형을 설정하고 시작시 오토스케일링 구성을 설정합니다.

In [ ]:
from time import gmtime, strftime

timestamp = strftime('-%Y-%m-%d-%H-%M-%S', gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))
print('You need this variable for cleaning resources\n\nendpoint_config_name=\"{}\"\n\n'.format(endpoint_config_name))

In [ ]:
timestamp = strftime('-%Y-%m-%d-%H-%M-%S', gmtime())
endpoint_name = job_name_prefix + '-ep' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}

## Endpoint 생성

In [ ]:
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

## Endpoint 상태 확인

> 대략 10~15분 정도 소요 됩니다.

In [ ]:
# get the status of the endpoint
response = sage.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))

## Endpoint 결과 확인

In [ ]:
%%time
try:
    # wait until the status has changed
    sage.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)
    # print the status of the endpoint
    endpoint_response = sage.describe_endpoint(EndpointName=endpoint_name)
    status = endpoint_response['EndpointStatus']
    print('Endpoint creation ended with EndpointStatus = {}'.format(status))
    print("\nUse endpoint variable next notebook\n\nendpoint_name = \"{}\"\n".format(endpoint_name))

    if status != 'InService':
        raise Exception('Endpoint creation failed.')
except:
    print('Training failed to start')
     # if exception is raised, that means it has failed
    message = sagemaker.describe_endpoint(EndpointName=endpoint_name)['FailureReason']
    print('Endpoint creation failed with the following error: {}'.format(message))